# Core 9 — Reservation Policy & Scheduler (core9_04)

본 노트북은 예측 모델을 만들지 않는다.

목적:
- core9_03에서 생성된 위험도(risk_score_total / hazard_level)를 입력으로 받아
- “Fallback 예약(Reservation)”을 **결정론적 규칙**으로 생성한다.

원칙:
- 예약은 fallback 실행이 아니다.
- 예약은 “곧 닫아야 하는가”를 미리 기록하는 운영 로그다.
- MIN_STEPS 이전에는 예약 확정(CONFIRMED)을 금지하고 HOLD로 기록한다.

산출물:
- core9_04_reservation_log.csv
- (선택) core9_04_reservation_params.json

In [2]:
from pathlib import Path
import json
import numpy as np
import pandas as pd

FORECAST_PATH = Path("../artifact/core9/core9_03_forecast_scores.csv")
FALLBACK_PARAMS_PATH = Path("../artifact/core8/core8_06_fallback_params.json")
FALLBACK_DECISIONS_PATH = Path("../artifact/core8/core8_06_fallback_decisions.csv")  # optional

assert FORECAST_PATH.exists(), "core9_03_forecast_scores.csv not found"
assert FALLBACK_PARAMS_PATH.exists(), "core8_06_fallback_params.json not found"

EXPORT_DIR = Path("../artifact/core9")
EXPORT_DIR.mkdir(exist_ok=True)

forecast = pd.read_csv(FORECAST_PATH)

with open(FALLBACK_PARAMS_PATH, "r", encoding="utf-8") as f:
    fallback_params = json.load(f)

forecast.head()

,run_id,case_id,antibody_id,step,risk_score_soms,risk_score_osc,risk_score_conflict,risk_score_total,forecast_hazard_level,forecast_rule_id,alarm_on,y_event_any,first_alarm_step,first_event_step,lead_time
0,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,0,0.000,0.0,NaN,NaN,NONE,core9_03_v1,False,0,NaN,NaN,NaN
1,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,1,0.000,0.0,NaN,NaN,NONE,core9_03_v1,False,0,NaN,NaN,NaN
2,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,2,0.000,0.0,NaN,NaN,NONE,core9_03_v1,False,0,NaN,NaN,NaN
3,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,3,0.000,0.0,NaN,NaN,NONE,core9_03_v1,False,0,NaN,NaN,NaN
4,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,4,0.006,0.0,NaN,NaN,NONE,core9_03_v1,False,0,NaN,NaN,NaN


In [3]:
RESERVATION_RULE_ID = "core9_04_v1"

# Core8 일관성: MIN_STEPS 재사용
MIN_STEPS = int(fallback_params.get("min_steps", 10))

# 예약 규칙 파라미터 (Core 9 핵심)
H = 5          # reservation horizon (해석/기록용)
K = 3          # HIGH 연속 K-step → CANDIDATE
M = 5          # CANDIDATE 이후 M-step 내 HIGH가 유지되면 → CONFIRMED
TTL = 10       # CONFIRMED 이후 TTL step 동안 유효, 이후 EXPIRED

# 위험도 기준: hazard_level 사용을 권장
HIGH_LEVEL = "HIGH"

PARAMS = {
    "reservation_rule_id": RESERVATION_RULE_ID,
    "min_steps": MIN_STEPS,
    "horizon_H": H,
    "K_consecutive_high_for_candidate": K,
    "M_window_for_confirm": M,
    "TTL_steps_after_confirm": TTL,
    "hazard_high_level": HIGH_LEVEL,
}
PARAMS # Reservation Policy Parameters

{'reservation_rule_id': 'core9_04_v1',
 'min_steps': 10,
 'horizon_H': 5,
 'K_consecutive_high_for_candidate': 3,
 'M_window_for_confirm': 5,
 'TTL_steps_after_confirm': 10,
 'hazard_high_level': 'HIGH'}

In [4]:
required_cols = [
    "run_id","case_id","antibody_id","step",
    "risk_score_total","forecast_hazard_level","forecast_rule_id"
]
missing = [c for c in required_cols if c not in forecast.columns]
assert not missing, f"Missing columns in forecast_scores: {missing}"

# dtype normalization
forecast["step"] = pd.to_numeric(forecast["step"], errors="coerce").astype("Int64")
forecast["risk_score_total"] = pd.to_numeric(forecast["risk_score_total"], errors="coerce")

forecast = forecast.sort_values(["run_id","case_id","antibody_id","step"]).reset_index(drop=True) # Forecast schema sanity check

In [5]:
def schedule_reservation_for_group(g: pd.DataFrame) -> pd.DataFrame:
    """
    입력: 하나의 trajectory (run_id, case_id, antibody_id 단위)
    출력: step별 reservation 상태 로그

    상태:
    - HOLD: step < MIN_STEPS → evaluated=True, confirmed 금지
    - NONE: 평가했으나 조건 미충족
    - CANDIDATE: HIGH가 K연속이면 후보 등록
    - CONFIRMED: 후보 이후 M-step 내 HIGH가 유지(정책적으로 ‘계속 HIGH’)되면 확정
    - EXPIRED: 확정 후 TTL 경과하면 만료
    """
    g = g.sort_values("step").copy()

    high = g["forecast_hazard_level"].fillna("NONE").eq(HIGH_LEVEL).to_numpy()
    steps = g["step"].astype(int).to_numpy()

    status = []
    reason = []
    evaluated = []

    # state variables
    consecutive_high = 0
    candidate_start_step = None
    confirmed_step = None
    expire_step = None

    for idx in range(len(g)):
        step = steps[idx]
        is_high = bool(high[idx])

        # 기본: 평가됨
        evaluated.append(True)

        # HOLD 구간: 예약 확정 금지
        if step < MIN_STEPS:
            status.append("HOLD")
            reason.append("REASON_MIN_STEPS_NOT_REACHED")
            continue

        # CONFIRMED 이후 TTL 처리
        if confirmed_step is not None:
            if step <= expire_step:
                status.append("CONFIRMED")
                reason.append("REASON_RESERVATION_CONFIRMED_TTL_ACTIVE")
            else:
                status.append("EXPIRED")
                reason.append("REASON_RESERVATION_TTL_EXPIRED")
            continue

        # 여기부터: step >= MIN_STEPS, confirmed 아직 없음
        if is_high:
            consecutive_high += 1
        else:
            consecutive_high = 0

        # CANDIDATE 생성
        if candidate_start_step is None and consecutive_high >= K:
            candidate_start_step = step - (K - 1)
            status.append("CANDIDATE")
            reason.append("REASON_CONSECUTIVE_HIGH_REACHED")
            continue

        # CANDIDATE 유지/확정 판단
        if candidate_start_step is not None:
            # 후보 이후 M-step 윈도우에서 "HIGH 유지"를 요구
            # 정책적 의미: 후보 등록 이후에도 위험이 꺼지지 않는가(비회복)
            # 구현: candidate_start_step ~ candidate_start_step + M 구간에서 계속 HIGH가 유지되는지
            window_end = candidate_start_step + (M - 1)

            if step <= window_end:
                # 후보 윈도우 기간 동안 HIGH가 한 번이라도 꺼지면 후보는 유지하되 "확정 금지"로 남김
                # (엄격 모드: 한 번이라도 꺼지면 후보 리셋하고 싶으면 아래를 수정하면 됨)
                status.append("CANDIDATE")
                if is_high:
                    reason.append("REASON_CANDIDATE_HIGH_MAINTAINED")
                else:
                    reason.append("REASON_CANDIDATE_HIGH_BROKEN")
                # window_end까지 관찰이 끝나기 전에는 확정 불가
                continue
            else:
                # 후보 관찰 윈도우 종료 후 확정 조건 체크:
                # candidate_start_step ~ window_end 구간이 모두 HIGH였으면 확정
                # (g에서 해당 구간의 HIGH가 모두 True인지 확인)
                mask = (steps >= candidate_start_step) & (steps <= window_end)
                all_high_in_window = bool(high[mask].all()) if mask.any() else False

                if all_high_in_window:
                    confirmed_step = step
                    expire_step = confirmed_step + TTL
                    status.append("CONFIRMED")
                    reason.append("REASON_RESERVATION_CONFIRMED")
                else:
                    # 확정 실패: 후보 해제하고 다시 관찰
                    candidate_start_step = None
                    status.append("NONE")
                    reason.append("REASON_CANDIDATE_NOT_SUSTAINED")
                continue

        # 후보도 확정도 아닌 경우
        status.append("NONE")
        reason.append("REASON_WITHIN_BOUNDS")

    out = g.copy()
    out["reservation_evaluated"] = evaluated
    out["reservation_status"] = status
    out["reservation_reason_code"] = reason
    out["reservation_horizon"] = H
    out["reservation_rule_id"] = RESERVATION_RULE_ID
    return out # Reservation state machine

In [6]:
reservation_log = (
    forecast
    .groupby(["run_id","case_id","antibody_id"], group_keys=False)
    .apply(schedule_reservation_for_group)
    .reset_index(drop=True)
)

reservation_log.head(30) # Run scheduler over all trajectories

if FALLBACK_DECISIONS_PATH.exists():
    fb = pd.read_csv(FALLBACK_DECISIONS_PATH)

    # Core8 decision columns (최소)
    keep = ["run_id","case_id","antibody_id","step","fallback_stage","fallback_reason_code","fallback_score"]
    keep = [c for c in keep if c in fb.columns]
    fb = fb[keep].copy()

    # duplicate columns 방지
    fb = fb.loc[:, ~fb.columns.duplicated()].copy()

    reservation_log = reservation_log.merge(
        fb,
        on=["run_id","case_id","antibody_id","step"],
        how="left",
        suffixes=("","_core8")
    )

reservation_log.head() # Optional: compare with Core8 fallback decisions

/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_21337/200297756.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(schedule_reservation_for_group)


,run_id,case_id,antibody_id,step,risk_score_soms,risk_score_osc,risk_score_conflict,risk_score_total,forecast_hazard_level,forecast_rule_id,...,first_event_step,lead_time,reservation_evaluated,reservation_status,reservation_reason_code,reservation_horizon,reservation_rule_id,fallback_stage,fallback_reason_code,fallback_score
0,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,0,0.000,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000
1,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,1,0.000,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000
2,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,2,0.000,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000
3,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,3,0.000,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000
4,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,4,0.006,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.006


In [7]:
# case-level 요약: 후보/확정/만료 존재 여부, 최초 step, 최대 위험도
tmp = reservation_log.copy()

tmp["is_candidate"] = tmp["reservation_status"].eq("CANDIDATE")
tmp["is_confirmed"] = tmp["reservation_status"].eq("CONFIRMED")
tmp["is_expired"]   = tmp["reservation_status"].eq("EXPIRED")
tmp["is_hold"]      = tmp["reservation_status"].eq("HOLD")

def first_step_where(g, cond_col):
    s = g.loc[g[cond_col], "step"]
    return int(s.min()) if len(s) else np.nan

rows = []
for (run_id, case_id), g in tmp.groupby(["run_id","case_id"]):
    rows.append({
        "run_id": run_id,
        "case_id": case_id,
        "has_hold": bool(g["is_hold"].any()),
        "has_candidate": bool(g["is_candidate"].any()),
        "has_confirmed": bool(g["is_confirmed"].any()),
        "has_expired": bool(g["is_expired"].any()),
        "first_candidate_step": first_step_where(g, "is_candidate"),
        "first_confirmed_step": first_step_where(g, "is_confirmed"),
        "max_risk_score_total": float(pd.to_numeric(g["risk_score_total"], errors="coerce").max()),
    })

case_summary = pd.DataFrame(rows)
case_summary

,run_id,case_id,has_hold,has_candidate,has_confirmed,has_expired,first_candidate_step,first_confirmed_step,max_risk_score_total
0,core7_04_1767776352,A_ALWAYS_ALLOW,True,False,False,False,NaN,NaN,0.120750
1,core7_04_1767776352,B_GOVERNED,True,False,False,False,NaN,NaN,0.413241


In [8]:
# export files
log_path = EXPORT_DIR / "core9_04_reservation_log.csv"
summary_path = EXPORT_DIR / "core9_04_case_summary.csv"
params_path = EXPORT_DIR / "core9_04_reservation_params.json"

reservation_log.to_csv(log_path, index=False)
case_summary.to_csv(summary_path, index=False)

with open(params_path, "w", encoding="utf-8") as f:
    json.dump(PARAMS, f, indent=2)

print("Exported:")
print("-", log_path)
print("-", summary_path)
print("-", params_path) # Export reservation log + params

Exported:
- ../artifact/core9/core9_04_reservation_log.csv
- ../artifact/core9/core9_04_case_summary.csv
- ../artifact/core9/core9_04_reservation_params.json


## Judgment Point

- Fallback은 반응이 아니라 “예약된 거버넌스 결정”입니다.
- 예약은 예측 점수 자체가 아니라, **결정론적 예약 규칙(policy)**로 생성됩니다.
- MIN_STEPS 이전에는 HOLD로 기록되어 “즉시 발동 금지”가 로그로 증명됩니다.
- CANDIDATE → CONFIRMED → EXPIRED 상태 전이는 운영 로그로 영속화되어 Core 10 회귀 조건으로 이식 가능합니다.